<a href="https://colab.research.google.com/github/michael18781/ComputationalPhys/blob/main/Exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3A: Helmholtz coils

Write a program to calculate the magnetic field caused by Helmholtz coils. Check your solution agrees
with the on‑axis analytical result. Investigate with suitable plots the uniformity of the field near the
centre of the system.

We have set the current always to be 1/mu_0 so that the Biot-Savart law becomes dB = (dl cross r)/(4pi r^3). Then to adjust for a different we just scale the B field as appropriate.

## Preliminary code

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math

R = 1.0
z = 0.0

def compute_dB(dl, position, x, y):
  """ 
  This code computes the contribution to the magnetic field (over a whole meshgrid) 
  due to a current element of length, dl and location, position. This is using
  the Biot-Savart law.
  """ 
  norm = np.sqrt((position[0] - x)**2 + (position[1] - y)**2 + (position[2] - z)**2)
  scaling = 1 / norm**3 * 1/(4*np.pi)

  dBx = (dl[1]*(position[2]-z)-dl[2]*(position[1]-y)) * scaling
  dBy = (dl[2]*(position[0]-x)-dl[0]*(position[2]-z)) * scaling
  return dBx, dBy

def compute_field(elements, x_max, y_max, nx, ny):
  """
  This piece of code will take in the meshgrid size and dimensions. Then
  it will loop over all the current elements and sum up their contributions giving
  the overall magnetic field. It will produce the field in 2D.
  """
  x = np.linspace(-x_max, x_max, nx)
  y = np.linspace(-y_max, y_max, ny)
  X, Y = np.meshgrid(x, y)

  Bx, By = np.zeros((ny, nx)), np.zeros((ny, nx))

  for element in elements:
    dBx, dBy = compute_dB(element[0], element[1], X, Y)
    Bx += dBx
    By += dBy

  # We return the linspaces and the magnetic field
  return x, y, Bx, By

def plot_streamlines(elements, x_max, y_max, nx, ny):
  """
  For a given set of current elements and size and dimensions of a grid, this 
  code will plot the magnetic field in the x-y plane in the form of streamlines.
  The brighter yellow the lines are, the stronger the magnetic field
  """
  x, y, Bx, By = compute_field(elements, x_max, y_max, nx, ny)

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)

  color = 2 * np.log(np.hypot(Bx, By)) # Colour depends on field strength
  # Plot the streamlines
  ax.streamplot(x, y, Bx, By, color=color, linewidth=1, cmap=plt.cm.hot, density=2, arrowstyle='->', arrowsize=1.5)

  # Plotting settings
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.set_xlim(-x_max, x_max)
  ax.set_ylim(-y_max, y_max)
  ax.set_aspect('equal')
  plt.show()

 Core task 1

This task is to calculate the magnetic field from a single coil of radius R = 1.0 and carrying a current 1/mu_0 centred on the origin. The code will produce a 2D plot of the magnetic field in the x, y plane. 

In [ ]:
def add_elements():
  elements = []
  n = 100
  for i in range(n):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([0, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

def plot_magnitude_against_x(x_max, y_max, nx, ny):
  """
  This function aims to plot the magnitude of the magnetic field along the x axis.
  It will take limits for the axis and then number of points along which to compute the 
  field. It will produce the same size and dimensions in the y direction. Then,
  it will compute the field and plot the magnitude of the field and the expected 
  field against x. It will plot the difference between the magnitude and the expected 
  field.
  """
  x, y, Bx, By = compute_field(add_elements(), x_max, y_max, nx, ny)

  magnitude = np.hypot(Bx, By)

  # We take all the points "half way up" the y-axis i.e. those along the x-axis
  magnitude_along_x = magnitude[:][int(ny/2)] 

  pred = R**2/(2*np.sqrt(x**2+R**2)**3) # The prediction based on theory

  fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 12))

  # Plotting absolute values of prediction and computation
  ax1.plot(x, magnitude_along_x, label='Computation')
  ax1.plot(x, pred, label='Theoretical prediction')

  # Plotting difference between prediction and computation
  ax2.plot(x, magnitude_along_x-pred, label='Difference in magnitude')

  ax1.legend()
  ax2.legend()

  ax1.set_xlabel('x')
  ax1.set_ylabel('Magnetic field / T')
  ax2.set_xlabel('x')
  ax2.set_ylabel('Difference between computation and prediction / T')
  plt.show()

plot_magnitude_against_x(10, 10, 500, 500)

In [ ]:
def add_elements():
  elements = []
  n = 100
  for i in range(n):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([0, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

plot_streamlines(add_elements(), 10, 10, 20, 20)

## Conclusions from core task 1

We first check the result agrees with that on axis. To check the magnitude, we can plot the magnitude of the magnetic field vs x:

![](https://drive.google.com/uc?export=view&id=1xi1QkgzylZXzxzBXyy0nXGdRezjLkKuX)

The difference between the expected field and the computed field is small and only noticeable near the peak at x = 0. This is likely due to the discrete sampling of the linspace and so we miss the effects of the rapidly changing magnetic field in this region. When increasing the number of sampling points in the linspace, we see that the absolute difference does decrease significantly, yet the shape of the difference curve remains the same.

We can see from this plot that the field is essentially in the x direction on the x axis:

![](https://drive.google.com/uc?export=view&id=13p94Xo1tnptdvAGDnki7Qmgm_UTqRICG)

 We can see a nice symmetry which we expect and the field lines pointing in the direction we expect. When we reverse the direction of the current by reversing the direction of the current elements, the field lines reverse in direction as expected.







# Core task 2

In this task we are going to produce a pair of Helmholtz coils and test the uniformity of the field between the coils.

In [ ]:
def add_elements():
  elements = []
  n = 100
  for i in range(n):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([-R/2, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([R/2, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

plot_streamlines(add_elements(), 2, 2, 20, 20)

In [ ]:
def add_elements():
  elements = []
  n = 100
  for i in range(n):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([-R/2, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([R/2, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

def max_pct_difference_in_10cm_cylinder(nx, ny):
  """
  This code will compute the magnetic field within a 10cm length, 10cm diameter
  cylinder centred on the origin with its axis in the x direction. Then it will
  find the maximum and minimum magnetic fields within it and compare to the field
  at the origin to find how uniform the field really is.
  """
  x, y, Bx, By = compute_field(add_elements(), 0.05, 0.05, nx, ny)

  magnitude = np.hypot(Bx, By) # Magnitude of the field
  B_at_origin = magnitude[int(nx/2)][int(ny/2)]

  maximum = np.max(magnitude) # Stronger field
  print((maximum-B_at_origin)/B_at_origin * 100.0)

  minimum = np.min(magnitude) # Weakest field
  print((minimum-B_at_origin)/B_at_origin * 100.0)

max_pct_difference_in_cylinder(20, 20)

## Conclusions from core task 2

We can see the uniformity of the magnetic field by plotting it around (0, 0, 0):

![](https://drive.google.com/uc?export=view&id=1IN5O76YDGUovg1YVSA6oWLQkcocfttYV)

Clearly, near the centre, the magnetic field lines are very straight and the same colour hence the field is approximately uniform. To quantify this further, we produce a plot between x = -0.05m and x = +0.05m (i.e. 10cm width) and y = -0.05m and y = +0.05m. This effectively gives us the cylinder projected into the x-y plane co-axial with the coils and of diameter 10cm and length 10cm. 

![](https://drive.google.com/uc?export=view&id=1vHWRkZsRu_1rwNwcWyLEpQWYKU7OrX9L)

The value for the field strength found at the origin is 0.7155 which agrees with the formula in the exercises manual. The largest field within the region is 0.0011% stronger that that at the origin. The smallest field within the region is 0.0007% weaker than that at the origin. These deviations are very small and hence the field is very uniform. The maximum percentage deviation of the field is thus 0.0011% (from the origin).

# Supplementary task

In this task we consider the effect on N coils together in a fixed length.

Firstly, we plot the magnitude of the magnetic field inside at the origin vs N. We expect this to be approximately linear because the field is equal to n (the coil density) for a solenoid with current = 1/mu_0. 

Secondly, we plot the difference between the computed field and the solenoid field at the origin vs N. We expect that for higher N, as the seperation of the coils reduces compared to the radius of the coils, the field will become more uniform and behave more like a solenoid. The approximation made for solenoids assumes they have a uniform field inside but this will start to apply only when the radius of the coils is much larger than their separation.

In [ ]:
D = 10*R

def add_elements(N):
  elements = []
  n = 100
  spacing = D/(N-1)

  for i in range(n):
    # A coil is added at multiple locations along the x-axis
    for j in range(N):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([-D/2+j*spacing, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

plot_streamlines(add_elements(10), D/2 + 1, D/2 + 1, 50, 50)

In [ ]:
D = 10*R

def add_elements(N):
  """
  This code adds N coils along the x-axis between +5R and -5R. This works for
  greater than one coil.
  """
  elements = []
  n = 100
  spacing = D/(N-1)

  for i in range(n):
    for j in range(N):
      elements.append(( np.array([0, -((2*np.pi*R)/n)*np.sin(2*np.pi*i/n), ((2*np.pi*R)/n)*np.cos(2*np.pi*i/n)]),
                        np.array([-D/2+j*spacing, R*np.cos(2*np.pi*i/n), R*np.sin(2*np.pi*i/n)]) ))
  return elements

def compare_number_of_coils(x_max, y_max, nx, ny):
  """
  This is some code to see how changing the number of coils in the region of space
  x = +5R to x = -5R with their axes along the x-axis affects the magnetic field
  at the origin and how this compares to the magnetic field produced by a solenoid.
  """
  N_max = 500
  ns = np.arange(30, N_max+1)
  field_strengths = np.zeros(len(ns))

  for (i, N) in enumerate(ns):
    x, y, Bx, By = compute_field(add_elements(N), x_max, y_max, nx, ny)
    field_strengths[i] = np.hypot(Bx, By)[int(nx/2)][int(ny/2)]

  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

  # Finding the difference in field strength between that computed and that expected
  # if we were observing a solenoid.
  pct_solenoidal = ((field_strengths - (ns - 1)/D)/((ns - 1)/D)) * 100.0

  # Difference from solenoid 
  ax1.plot(ns, pct_solenoidal)
  ax1.set_xlabel('Number of coils')
  ax1.set_ylabel('Pct. diff. between computed field strength and expected')

  # The field strength vs N
  ax2.plot(ns, field_strengths)
  ax2.set_xlabel('Number of coils')
  ax2.set_ylabel('Field strength at the origin / T')

compare_number_of_coils(10, 10, 20, 20)

## Conclusions from supplementary task
Firstly with 10 coils:

![](https://drive.google.com/uc?export=view&id=1KGu_rFKF7Sdbk4UCLiX7wgjuFwxm4Wm6)

This shows us that the magnetic field outside the coils is very weak (the dark red and black lines) yet the field inside is strong and uniform.

We expect to see that the magnetic field strength is approximately proportional to the number density of coils and hence (as the length is fixed) the number of coils, N.

We expect that the difference between the magnetic field strength computed and the expectation (assuming it is a solenoid, B is proportional to the density of coils) decreases as the number of coils increases. This is because the approximation that the field is uniform within the solenoid becomes more accurate. 

![](https://drive.google.com/uc?export=view&id=13ufdnJKRtRCBmxiO1vMBFz8mtWT8zNBJ)

This is what we observe and it agrees with our expectations. For low N, the solenoid approximately is clearly very poor and should not be used. However, as we increase N, the spacing between coils reduces and we see that the computed magnetic field tends to what we expect for a solenoid. The plot of magnetic field vs. N shows approximately linear behaviour as expected.